# Generate annual/yearly zarr stores from hourly ERA5 NetCDF files on NCAR's Geoscience Data Exchange


## Overview and Warning: Please Read
- ERA5 data on NCAR's Geoscience Data Exchange (GDEX) is stored in hourly NetCDF files. Therefore, it is necessary to create intermediate ARCO datasets for fast processing.
- In this notebook, we read hourly data from NCAR's publicly accessible ERA5 collection using an intake catalog, compute the annual means and store the result using zarr stores.
- If you don't have write permision to save to NCAR's GDEX, please save the result to your local folder.
- If you need annual means for the following variables, please don't run the commented cells in this notebook. The data has already been calculated and can be accessed via https from https://data.gdex.ucar.edu/special_projects/pythia_2024/annual_means/

  1) Air temperature at 2 m/ [VAR_2T](https://data.gdex.ucar.edu/special_projects/pythia_2024/annual_means/temp_2m_annual_1940_2023.zarr)
     
- Otherwise, please run this script once to generate the annual means.

## Prerequisites

| Concepts | Importance | Notes |
| --- | --- | --- |
| [Intro to Xarray](https://foundations.projectpythia.org/core/xarray) | Necessary | |
| [Intro to Intake](https://projectpythia.org/intake-cookbook/notebooks/intake_introduction) | Necessary | |
| [Understanding of Zarr](https://zarr.readthedocs.io/en/stable/) | Helpful | |

- **Time to learn**: 30 minutes

## Imports

In [1]:
import glob
import re
import matplotlib as plt
import numpy as np
import scipy as sp
import xarray as xr
import intake
import intake_esm
import pandas as pd

In [2]:
import dask
from dask.distributed import Client, performance_report,LocalCluster
from dask_jobqueue import PBSCluster

In [3]:
######## File paths ################
gdex_scratch   = "/lustre/desc1/scratch/harshah"
era5_catalog = 'https://data.gdex.ucar.edu/special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json'
annual_means      = '/gdex/data/special_projects/pythia_2024/annual_means/' #Posix path, only used for writing data 
annual_means_https= 'https://data.gdex.ucar.edu/special_projects/pythia_2024/annual_means/'

print(era5_catalog)

https://data.gdex.ucar.edu/special_projects/pythia_2024/pythia_intake_catalogs/era5_catalog.json


## Create a Dask cluster

#### Dask Introduction

[Dask](https://www.dask.org/) is a solution that enables the scaling of Python libraries. It mimics popular scientific libraries such as numpy, pandas, and xarray that enables an easier path to parallel processing without having to refactor code. 

There are 3 components to parallel processing with Dask: the client, the scheduler, and the workers. 

The Client is best envisioned as the application that sends information to the Dask cluster. In Python applications this is handled when the client is defined with `client = Client(CLUSTER_TYPE)`. A Dask cluster comprises of a single scheduler that manages the execution of tasks on workers. The `CLUSTER_TYPE` can be defined in a number of different ways.

- There is LocalCluster, a cluster running on the same hardware as the application and sharing the available resources, directly in Python with `dask.distributed`. 

- In certain JupyterHubs Dask Gateway may be available and a dedicated dask cluster with its own resources can be created dynamically with `dask.gateway`. 

- On HPC systems `dask_jobqueue` is used to connect to the HPC Slurm and PBS job schedulers to provision resources.

The `dask.distributed` client python module can also be used to connect to existing clusters. A Dask Scheduler and Workers can be deployed in containers, or on Kubernetes, without using a Python function to create a dask cluster. The `dask.distributed` Client is configured to connect to the scheduler either by container name, or by the Kubernetes service name.   

#### Select the Dask cluster type

The default will be `LocalCluster` as that can run on any system.

If running on a HPC computer with a PBS Scheduler, set to True. Otherwise, set to False.

In [4]:
USE_PBS_SCHEDULER = False

If running on Jupyter server with Dask Gateway configured, set to True. Otherwise, set to False.

In [5]:
USE_DASK_GATEWAY = False

**Python function for a PBS cluster**

In [6]:
# Create a PBS cluster object
def get_pbs_cluster():
    """ Create cluster through dask_jobqueue.   
    """
    from dask_jobqueue import PBSCluster
    cluster = PBSCluster(
        job_name = 'dask-pythia-24',
        cores = 1,
        memory = '4GiB',
        processes = 1,
        local_directory = gdex_scratch + '/dask/spill',
        resource_spec = 'select=1:ncpus=1:mem=8GB',
        queue = 'casper',
        walltime = '1:00:00',
        #interface = 'ib0'
        interface = 'ext'
    )
    return cluster

**Python function for a Gateway Cluster**

In [7]:
def get_gateway_cluster():
    """ Create cluster through dask_gateway
    """
    from dask_gateway import Gateway

    gateway = Gateway()
    cluster = gateway.new_cluster()
    cluster.adapt(minimum=2, maximum=4)
    return cluster

**Python function for a Local Cluster**

In [8]:
def get_local_cluster():
    """ Create cluster using the Jupyter server's resources
    """
    from distributed import LocalCluster, performance_report
    cluster = LocalCluster()    
    cluster.scale(6)
    return cluster

**Python logic to select the Dask Cluster type**

This uses True/False boolean logic based on the variables set in the previous cells

In [9]:
# Obtain dask cluster in one of three ways

if USE_PBS_SCHEDULER:
    cluster = get_pbs_cluster()
elif USE_DASK_GATEWAY:
    cluster = get_gateway_cluster()
else:
    cluster = get_local_cluster()

# Connect to cluster
from distributed import Client
client = Client(cluster)

# Display cluster dashboard URL
cluster

LocalCluster(5fb129b6, 'tcp://127.0.0.1:37621', workers=6, threads=24, memory=22.73 GiB)

## Find data using intake catalog 

In [10]:
era5_cat = intake.open_esm_datastore(era5_catalog)
era5_cat

/srv/conda/envs/notebook/lib/python3.10/site-packages/intake_esm/cat.py:250: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,unique
era_id,1
datatype,2
level_type,1
step_type,7
table_code,4
param_code,164
variable,212
long_name,212
units,33
year,85


In [11]:
era5_cat.df[['long_name','variable']].drop_duplicates().head()

,long_name,variable
0,Potential vorticity,PV
31,Specific rain water content,CRWC
62,Specific snow water content,CSWC
93,Geopotential,Z
124,Temperature,T


### Search for variable of interest

:::{hint}
- Examples of searching for other variables 
  1. olr_cat = era5_cat.search(variable ='MTNLWRF'), where MTNLWRF = Outgoing Long Wave Radiation (upto a sign),        Mean Top Net Long Wave Radiative Flux
  2. rh_cat = era5_cat.search(variable= 'R') # R =  Relative Humidity
:::

In [12]:
############ Access temperature data ###########
temp_cat = era5_cat.search(variable='VAR_2T',frequency = 'hourly')
temp_cat

,unique
era_id,1
datatype,1
level_type,0
step_type,1
table_code,1
param_code,1
variable,1
long_name,1
units,1
year,85


In [13]:
# Define the xarray_open_kwargs with a compatible engine, for example, 'scipy'
xarray_open_kwargs = {
    'engine': 'h5netcdf',
    #'engine': 'netcdf4',
    'chunks': {},  # Specify any chunking if needed
    'backend_kwargs': {}  # Any additional backend arguments if required
}

In [14]:
%%time
dset_temp = temp_cat.to_dataset_dict(xarray_open_kwargs=xarray_open_kwargs)


--> The keys in the returned dictionary of datasets are constructed as follows:
	'datatype.step_type'


CPU times: user 3min 58s, sys: 9.12 s, total: 4min 7s
Wall time: 7min 25s


In [15]:
dset_temp.keys()

dict_keys(['an.sfc'])

In [16]:
temp_2m = dset_temp['an.sfc'].VAR_2T
temp_2m

<xarray.DataArray 'VAR_2T' (time: 737784, latitude: 721, longitude: 1440)> Size: 3TB
dask.array<concatenate, shape=(737784, 721, 1440), dtype=float32, chunksize=(27, 139, 277), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 6MB 1940-01-01 ... 2024-02-29T23:00:00
    utc_date   (time) int32 3MB dask.array<chunksize=(744,), meta=np.ndarray>
Attributes: (12/14)
    long_name:                                          2 metre temperature
    short_name:                                         2t
    units:                                              K
    original_format:                                    WMO GRIB 1 with ECMWF...
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    ...                                                 ...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    number_of_significant_digits:                       7
    QuantizeBitGroomNumberOfSignificantDigits:          7

### Resample hourly data to annual data and save 

:::{warning}
Uncomment and run the cells in this sub-section only if you want to recalculate the annual means! This is a computationally expensive step.
:::

In [17]:
# temp_2m_annual = temp_2m.resample(time='YE').mean()
# temp_2m_annual

In [18]:
# temp_2m_annual = temp_2m_annual.chunk({'latitude':721,'longitude':1440})
# temp_2m_annual = temp_2m_annual.drop_isel({'time':-1}) # Drop 2024 data
# temp_2m_annual

- Save annual mean to annual_means folder within gdex_data
- You may not have permissions to write to this folder. 
- If so, please replace the folder path in the cell below with your own path where you have write permissions


In [19]:
# %%time
# temp_2m_annual.to_dataset().to_zarr(annual_means + 'temp_2m_annual_1940_2023.zarr',mode='w')

### Open the saved file and make a test plot

In [20]:
temp_2m_annual = xr.open_zarr(annual_means_https + 'temp_2m_annual_1940_2023.zarr').VAR_2T
temp_2m_annual 

<xarray.DataArray 'VAR_2T' (time: 84, latitude: 721, longitude: 1440)> Size: 349MB
dask.array<open_dataset-VAR_2T, shape=(84, 721, 1440), dtype=float32, chunksize=(84, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 6kB 90.0 89.75 89.5 ... -89.5 -89.75 -90.0
  * longitude  (longitude) float64 12kB 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
  * time       (time) datetime64[ns] 672B 1940-12-31 1941-12-31 ... 2023-12-31
Attributes: (12/14)
    QuantizeGranularBitGroomNumberOfSignificantDigits:  7
    ecmwf_local_table:                                  128
    ecmwf_parameter:                                    167
    grid_specification:                                 0.25 degree x 0.25 de...
    long_name:                                          2 metre temperature
    maximum_value:                                      320.42938232421875
    ...                                                 ...
    rda_dataset:                                        ds633.0
    rda_dataset_doi:                                    DOI: 10.5065/BH6N-5N20
    rda_dataset_group:                                  ERA5 atmospheric surf...
    rda_dataset_url:                                    https:/rda.ucar.edu/d...
    short_name:                                         2t
    units:                                              K

In [1]:
%%time
temp_2m_annual.isel(time=0).plot()

NameError: name 'temp_2m_annual' is not defined

### Close up the cluster

In [ ]:
cluster.close()